In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_BESTCASE_RESIDUAL, run_GaBP_HARDWARE_BESTCASE, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [2]:
# import random

# def custom_sample(items, n):
#     indices = random.sample(range(len(items)), n)
#     return [items[i] for i in indices]

# # Example usage
# items = ["apple", "banana", "orange", "grape", "kiwi"]
# n = 10
# selected_items = custom_sample(items, n)
# print(selected_items)

In [3]:
num_nodes = 100

sync_convergence_threshold = 1*10**-5

NODE_UPDT_PE = 20
PEs = 1

#### Sync

In [4]:
sum_of_iterations = 0
num_iterations = 1

TRIES = 100_000
ASYNC_ITER = 10

for _ in range(TRIES):
    A, b = data_gen.get_1D_line_matrix(num_nodes, scaling=True, normalized=False)
    # A, b = data_gen.get_2D_lattice_matrix(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)))
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=100_000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        iteration = float("inf")
        print("=========== RESTART ===========")

print(f"NUMBER SYNC ITERATIONS = {iteration}")

iteration: 1
1.13342518560285
-----
iteration: 2
0.3567170218513065
-----
iteration: 3
0.23103100964230655
-----
iteration: 4
0.5874443082581844
-----
iteration: 5
0.595328533434388
-----
iteration: 6
0.4411013165489065
-----
iteration: 7
0.22141199902368286
-----
iteration: 8
0.17510092019398818
-----
iteration: 9
0.11209519281769928
-----
iteration: 10
0.15071754406794746
-----
iteration: 11
0.16174518808563745
-----
iteration: 12
0.03444510793017035
-----
iteration: 13
0.03928410897623224
-----
iteration: 14
0.025538254795085376
-----
iteration: 15
0.04668096300679627
-----
iteration: 16
0.03452863501303974
-----
iteration: 17
0.03549393941119235
-----
iteration: 18
0.0022889119499475584
-----
iteration: 19
0.012936222938468526
-----
iteration: 20
0.004718049570424752
-----
iteration: 21
0.005683478207641629
-----
iteration: 22
0.00651686648291927
-----
iteration: 23
0.0010565873674009939
-----
iteration: 24
0.0011782443331175077
-----
iteration: 25
0.0013236435464950914
-----
itera

#### Random - No Cache

In [5]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE/2, number_pes=2*PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 148 --------------
-------------- ITERATION = 2 => Streams = 186 --------------
-------------- ITERATION = 3 => Streams = 152 --------------
-------------- ITERATION = 4 => Streams = 168 --------------
-------------- ITERATION = 5 => Streams = 141 --------------
-------------- ITERATION = 6 => Streams = 132 --------------
-------------- ITERATION = 7 => Streams = 147 --------------
-------------- ITERATION = 8 => Streams = 121 --------------
-------------- ITERATION = 9 => Streams = 135 --------------
-------------- ITERATION = 10 => Streams = 161 --------------
AVE. ASYNC ITERATIONS = 149.1
-------------- ITERATION = 1 => Streams = 125 --------------
-------------- ITERATION = 2 => Streams = 146 --------------
-------------- ITERATION = 3 => Streams = 140 --------------
-------------- ITERATION = 4 => Streams = 163 --------------
-------------- ITERATION = 5 => Streams = 169 --------------
-------------- ITERATION = 6 => Streams = 159 --------

#### Random - with Cache

In [6]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")


async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE/2, number_pes=PEs*2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 123 --------------
-------------- ITERATION = 2 => Streams = 114 --------------
-------------- ITERATION = 3 => Streams = 110 --------------
-------------- ITERATION = 4 => Streams = 135 --------------
-------------- ITERATION = 5 => Streams = 115 --------------
-------------- ITERATION = 6 => Streams = 113 --------------
-------------- ITERATION = 7 => Streams = 109 --------------
-------------- ITERATION = 8 => Streams = 175 --------------
-------------- ITERATION = 9 => Streams = 160 --------------
-------------- ITERATION = 10 => Streams = 141 --------------
AVE. ASYNC ITERATIONS = 129.5
-------------- ITERATION = 1 => Streams = 147 --------------
-------------- ITERATION = 2 => Streams = 116 --------------
-------------- ITERATION = 3 => Streams = 133 --------------
-------------- ITERATION = 4 => Streams = 131 --------------
-------------- ITERATION = 5 => Streams = 126 --------------
-------------- ITERATION = 6 => Streams = 140 --------

#### Random (Exclusion) - No Cache

In [21]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

it = 0

while (it < num_iterations):

    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100_000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    
    if iteration > 0:
        sum_of_iterations += iteration
        print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
        it += 1

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 217 --------------
-------------- ITERATION = 2 => Streams = 281 --------------
-------------- ITERATION = 3 => Streams = 239 --------------
-------------- ITERATION = 4 => Streams = 243 --------------
-------------- ITERATION = 5 => Streams = 265 --------------
-------------- ITERATION = 6 => Streams = 219 --------------
-------------- ITERATION = 7 => Streams = 239 --------------
-------------- ITERATION = 8 => Streams = 264 --------------
-------------- ITERATION = 9 => Streams = 208 --------------
-------------- ITERATION = 10 => Streams = 224 --------------
AVE. ASYNC ITERATIONS = 239.9


#### Random (Exclusion) - With Cache

In [22]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 224 --------------
-------------- ITERATION = 2 => Streams = 272 --------------
-------------- ITERATION = 3 => Streams = 214 --------------
-------------- ITERATION = 4 => Streams = 203 --------------
-------------- ITERATION = 5 => Streams = 253 --------------
-------------- ITERATION = 6 => Streams = 174 --------------
-------------- ITERATION = 7 => Streams = 168 --------------
-------------- ITERATION = 8 => Streams = 192 --------------
-------------- ITERATION = 9 => Streams = 197 --------------
-------------- ITERATION = 10 => Streams = 201 --------------
AVE. ASYNC ITERATIONS = 209.8


#### Residual - with Cache

In [23]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 81 --------------
-------------- ITERATION = 2 => Streams = 95 --------------
-------------- ITERATION = 3 => Streams = 90 --------------
-------------- ITERATION = 4 => Streams = 91 --------------
-------------- ITERATION = 5 => Streams = 79 --------------
-------------- ITERATION = 6 => Streams = 79 --------------
-------------- ITERATION = 7 => Streams = 86 --------------
-------------- ITERATION = 8 => Streams = 88 --------------
-------------- ITERATION = 9 => Streams = 93 --------------
-------------- ITERATION = 10 => Streams = 88 --------------
AVE. ASYNC ITERATIONS = 87.0


#### Residual - with Cache

In [24]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 76 --------------
-------------- ITERATION = 2 => Streams = 73 --------------
-------------- ITERATION = 3 => Streams = 77 --------------
-------------- ITERATION = 4 => Streams = 84 --------------
-------------- ITERATION = 5 => Streams = 74 --------------
-------------- ITERATION = 6 => Streams = 76 --------------
-------------- ITERATION = 7 => Streams = 76 --------------
-------------- ITERATION = 8 => Streams = 76 --------------
-------------- ITERATION = 9 => Streams = 76 --------------
-------------- ITERATION = 10 => Streams = 78 --------------
AVE. ASYNC ITERATIONS = 76.6


#### Fixed - No Caching

In [25]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 132 --------------
-------------- ITERATION = 2 => Streams = 137 --------------
-------------- ITERATION = 3 => Streams = 147 --------------
-------------- ITERATION = 4 => Streams = 131 --------------
-------------- ITERATION = 5 => Streams = 128 --------------
-------------- ITERATION = 6 => Streams = 145 --------------
-------------- ITERATION = 7 => Streams = 136 --------------
-------------- ITERATION = 8 => Streams = 120 --------------
-------------- ITERATION = 9 => Streams = 140 --------------
-------------- ITERATION = 10 => Streams = 125 --------------
AVE. ASYNC ITERATIONS = 134.1


### Fixed - Cache

In [26]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 104 --------------
-------------- ITERATION = 2 => Streams = 121 --------------
-------------- ITERATION = 3 => Streams = 119 --------------
-------------- ITERATION = 4 => Streams = 115 --------------
-------------- ITERATION = 5 => Streams = 110 --------------
-------------- ITERATION = 6 => Streams = 114 --------------
-------------- ITERATION = 7 => Streams = 113 --------------
-------------- ITERATION = 8 => Streams = 126 --------------
-------------- ITERATION = 9 => Streams = 110 --------------
-------------- ITERATION = 10 => Streams = 120 --------------
AVE. ASYNC ITERATIONS = 115.2


### NO STREAMS (BESTCASE)

In [27]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### NO STREAMS (RESIDUAL)

In [28]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE_RESIDUAL(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=500, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")